
# **Asignatura de Big Data Analytics**

Prof. José M. Luna

jmluna@uco.es

---

En primer lugar, debemos instalar todas las dependencias necesarias para poder ejecutar el algoritmo. Así, tenemos que instalar Java 8, Spark 3.0 con Hadoop 3.2, y findspark para alojar spark en el sistema (acceder a Spark desde un Notebook).

In [1]:
# Instalacion de java
# !apt-get install openjdk-8-jdk-headless -qq > /dev/null

# Instalacion de spark
#!wget -q https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop3.2.tgz

# Descomprimimos el fichero de spark
#!tar xf spark-3.0.0-bin-hadoop3.2.tgz

The system cannot find the path specified.


In [ ]:
# Instalamos findspark
#!pip install -q findspark

In [4]:
# Establecemos las variables de entorno para Java y Spark
import os
#os.environ["JAVA_HOME"] = "C:\Program Files\Java\jdk-18.0.1.1"
#os.environ["SPARK_HOME"] = "C:\Program Files\Spark\spark-3.0.0-bin-hadoop3.2"

Ejecutamos una sesion local de spark para comprobar que esta funcionando.

In [5]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

Exception: Java gateway process exited before sending its port number

Importamos el conjunto de datos con el que queremos trabajar y comprobamos que se ha subido correctamente

In [ ]:
from google.colab import files
uploaded = files.upload()

!ls

Saving BostonHousing.csv to BostonHousing.csv
BostonHousing.csv  sample_data	spark-3.0.0-bin-hadoop3.2  spark-3.0.0-bin-hadoop3.2.tgz


Leemos el dataset con el que queremos trabajar, en este caso vamos a trabajar con el dataset de BostonHousing



In [ ]:
dataset = spark.read.csv('BostonHousing.csv',inferSchema=True, header =True)

Mostramos los atributos que tiene el dataset y su tipo

In [ ]:
dataset.printSchema()

root
 |-- crim: double (nullable = true)
 |-- zn: double (nullable = true)
 |-- indus: double (nullable = true)
 |-- chas: integer (nullable = true)
 |-- nox: double (nullable = true)
 |-- rm: double (nullable = true)
 |-- age: double (nullable = true)
 |-- dis: double (nullable = true)
 |-- rad: integer (nullable = true)
 |-- tax: integer (nullable = true)
 |-- ptratio: double (nullable = true)
 |-- b: double (nullable = true)
 |-- lstat: double (nullable = true)
 |-- medv: double (nullable = true)



En primer lugar, vamos a trabajar con una sola variable y una salida. Vamos a coger, por ejemplo, la variable crim y la varible medv (salida)

In [ ]:
subdataset = dataset.select("crim","medv")
subdataset.printSchema()
subdataset.show()

root
 |-- crim: double (nullable = true)
 |-- medv: double (nullable = true)

+-------+----+
|   crim|medv|
+-------+----+
|0.00632|24.0|
|0.02731|21.6|
|0.02729|34.7|
|0.03237|33.4|
|0.06905|36.2|
|0.02985|28.7|
|0.08829|22.9|
|0.14455|27.1|
|0.21124|16.5|
|0.17004|18.9|
|0.22489|15.0|
|0.11747|18.9|
|0.09378|21.7|
|0.62976|20.4|
|0.63796|18.2|
|0.62739|19.9|
|1.05393|23.1|
| 0.7842|17.5|
|0.80271|20.2|
| 0.7258|18.2|
+-------+----+
only showing top 20 rows



Creamos un nuevo conjunto de datos donde los inputs serán un vector (necesario para LinearRegression), pero consideramos sólo un atributo

In [ ]:
from pyspark.ml.feature import VectorAssembler

df = VectorAssembler(inputCols=['crim'], outputCol='inputs').transform(subdataset)

df = df.select(['inputs','medv'])

df.printSchema()

root
 |-- inputs: vector (nullable = true)
 |-- medv: double (nullable = true)



Dividimos el conjunto de datos en train y test. Vamos a considerar una partición del tipo 80/20

In [ ]:
trainData,testData = df.randomSplit([0.8,0.2])

Indicamos qué atributos serán las entradas y cuales serán la salida. En este caso, vamos a tomar como entrada "crim" y como salida "medv". Finalmente, entrenamos y testeamos con trainData y testData, y mostramos las predicciones

In [ ]:
from pyspark.ml.regression import LinearRegression

regressor = LinearRegression(featuresCol = 'inputs', labelCol = 'medv')

#Entrenamos el modelo con el conjunto de train
regressor = regressor.fit(trainData)

#Evaluamos el modelo con el conjunto de test
pred = regressor.evaluate(testData)

#Mostramos las predicciones
pred.predictions.show()

+---------+----+------------------+
|   inputs|medv|        prediction|
+---------+----+------------------+
|[0.01778]|32.9|23.907667518087084|
|[0.02187]|31.1| 23.90609520199548|
|[0.03041]|18.5|23.902812175241863|
|[0.03113]|17.5|23.902535386101043|
| [0.0315]|34.9| 23.90239314723701|
|[0.03359]|34.9| 23.90158968986991|
|[0.03445]|24.1|23.901259080618374|
|[0.03659]|24.8|23.900436401783157|
|[0.03705]|35.4|23.900259564276524|
|[0.03932]|22.0|23.899386909624216|
|[0.04203]|22.9|23.898345106052517|
|[0.04294]|20.6| 23.89799527533287|
|[0.05023]|17.1|23.895192785282067|
|[0.05059]|23.9|23.895054390711657|
|[0.05188]|22.5|23.894558476834355|
| [0.0536]|25.0|23.893897258331286|
|[0.05425]|24.6| 23.89364737924582|
| [0.0578]|37.2|23.892282655009833|
|[0.06047]|29.6|23.891256228612626|
|[0.06076]|23.9|23.891144744097573|
+---------+----+------------------+
only showing top 20 rows



Mostramos resultados del modelo de regresión en train

In [ ]:
#coeficientes del modelo de regresion
coeff = regressor.coefficients
#interseccioon del model con Y
intr = regressor.intercept
print ("Coeficientes: %a" %coeff)
print ("Interseccion: %f" %intr)

#Entrenamiento
print("Root Mean Squared Err (RMSE): %f" % regressor.summary.rootMeanSquaredError)
print("Mean Absolute Err (MAE): %f" % regressor.summary.meanAbsoluteError)
print("Mean Squared Err (MSE): %f" % regressor.summary.meanSquaredError)

Coeficientes: DenseVector([-0.3844])
Interseccion: 23.914503
Root Mean Squared Err (RMSE): 8.630452
Mean Absolute Err (MAE): 6.276315
Mean Squared Err (MSE): 74.484702


Mostramos resultados del modelo de regresión en test

In [ ]:
from pyspark.ml.evaluation import RegressionEvaluator

rmse = RegressionEvaluator(labelCol="medv", predictionCol="prediction", metricName="rmse").evaluate(pred.predictions)
print ("Root Mean Squared Err (RMSE): %f" % rmse)
mae = RegressionEvaluator(labelCol="medv", predictionCol="prediction", metricName="mae").evaluate(pred.predictions)
print ("Mean Absolute Err (MAE): %f" % mae)
mse = RegressionEvaluator(labelCol="medv", predictionCol="prediction", metricName="mse").evaluate(pred.predictions)
print ("Mean Squared Err (MSE): %f" % mse)

Root Mean Squared Err (RMSE): 7.782148
Mean Absolute Err (MAE): 5.525428
Mean Squared Err (MSE): 60.561829
